![Logo UOC](https://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/logotips/logo-UOC-2linies.png)

# TFG - Inteligencia Artificial
Enero de 2026

## Predicción de Respuesta a Tratamientos Oncológicos Basada en el Perfil Genético Mediante Técnicas de Aprendizaje Automático (ML) e Identificación de Genes Candidatos a Biomarcador Usando Técnicas de Explicabilidad (XAI) y Cuantificación de la Incertidumbre (UQ)

---

#### Pablo Vázquez Rodríguez
##### Grado en Ingeniería Informática
##### Inteligencia Artificial

#### Dra. María Moreno de Castro
#### Dr. Friman Sánchez

---


### Composición del dataset inicial
Desde la web CTR-DB se descargan los datos de cada estudio por separado y aquí simplemente se unen para crear un dataset único sobre el que trabajar.

In [1]:
# Importaciones
import pandas as pd
from pathlib import Path
from git import Repo  # Para clonar el repositorio con los datos no unificados

In [2]:
# Configuración
storage_repo_url = "https://github.com/pvazquezr/TFG_AI_STORAGE"
local_tmp_folder = Path(f"/tmp/TFG_AI_STORAGE")
unstructured_data_folder_name = "CTR_Microarray_Complete_Unstructured_dataset"
unstructured_data_folder_path = Path(f"{local_tmp_folder}/{unstructured_data_folder_name}")

In [3]:
# Función auxiliar para cargar y procesar cada matriz de expresión
def load_expression_matrix(path):
    df = pd.read_csv(path, index_col=0, decimal=',')
    return df

# Función auxiliar para cargar sólo los metadatos necesarios por
# cada estudio
def load_metadata(path):
    meta = pd.read_csv(path)
    return meta[['Sample_id', 'Response']]

In [4]:
# Clonación del repositorio de almacenamiento de los datos no unificados
if local_tmp_folder.exists():
    print("Actualizando repositorio...", end="")
    repo = Repo(local_tmp_folder)
    repo.remotes.origin.pull()
    print(" Hecho!")
else:
    print("Clonando repositorio...", end="")
    repo = Repo.clone_from(storage_repo_url, local_tmp_folder)
    print(" Hecho")

Clonando repositorio... Hecho


In [5]:
# Lista para almacenar todas las matrices formateadas
formatted_matrix_list = []

# Formatear todas las matrices para incluir respuesta y transponerlas
for folder in unstructured_data_folder_path.iterdir():
    print(f"Procesando {folder} ...", end="")

    # Leer matriz de expresión y metadatos
    expression_matrix = load_expression_matrix(f"{folder}/matrix_.csv")
    metadata = load_metadata(f"{folder}/cli.inf_.csv")
    
    # Transponer para que cada fila corresponda a una muestra
    expression_matrix = expression_matrix.T
    expression_matrix['Sample_id'] = expression_matrix.index
    
    # Añadir la respuesta del tratamiento según corresponde
    formatted_expression_matrix = pd.merge(expression_matrix, metadata, on='Sample_id')

    # Guardar la matriz formateada en la lista
    formatted_matrix_list.append(formatted_expression_matrix)

    print(" Hecho!")

Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_1-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_10-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_100-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_102-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_104-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_106-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_107-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_120-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_129-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/

In [6]:
# Unir todos los datasets
expression_df = pd.concat(formatted_matrix_list, ignore_index=True)
print(expression_df.shape)

(1889, 26011)


In [7]:
# Eliminar todos los genes que no tengan datos de expresión
# para todas las muestras

# Hacemos primero un double check para asegurarnos de que
# no se haya colado ningún campo vacío en Sample_id ni Response
empty_sample_id = expression_df["Sample_id"].isna().any()
empty_response_id = expression_df["Response"].isna().any()

if empty_sample_id or empty_response_id:
    raise Exception("HAY DATOS VACÍOS EN SAMPLE_ID O RESPONSE. PROCESO ABORTADO")
else:
    # Ahora con seguridad podemos eliminar los genes (columnas) con
    # algún dato vacío
    final_expression_df = expression_df.dropna(axis=1, how="any")

In [9]:
print(final_expression_df.shape)
final_expression_df.head()

(1889, 2396)


,ABCA5,ABCC3,ABCC5,ABCF1,ABCG1,ABHD2,ABHD4,ABI2,ABL2,ABTB2,...,ZNF821,ZNF862,ZNHIT2,ZSCAN18,ZSWIM1,ZWILCH,ZXDC,ZYX,Sample_id,Response
0,4.2238037533743,5.79683455536566,6.18921066913001,4.99750789506527,5.06198980397518,6.70198177320029,4.46301850164785,6.97690469497582,7.35393491840359,4.67175432039558,...,5.70189564890264,5.19908584519563,3.49272347839646,3.2765360020004,3.94482328034395,5.78397252765549,5.94475080840537,6.36294674429626,GSM1233067,Non_response
1,4.65524084867857,4.88859828629659,5.04749936909492,5.62285225876584,5.88365449974086,7.06882025018556,3.59552742593181,7.59372643361445,6.61432888865345,4.83942430752995,...,5.76584311179437,6.40955445840396,3.75004785660275,4.46232651650123,4.0130773296742,5.14234461901744,6.32153240664303,7.26577065664173,GSM1233069,Non_response
2,4.20307781995411,5.87137344877555,5.78483413401142,5.62843473670729,5.54452052407153,7.62823947259486,3.60758646348901,7.48953938692363,7.63467007436116,4.63280318550203,...,5.55381763685768,5.47380776390632,4.2501980827799,4.6161302183747,3.67684002621906,4.9734291047121,6.89797147825129,7.24794805956634,GSM1233072,Non_response
3,4.5062616916709,6.66059695914594,6.19373297740869,6.24588339162663,5.40567492861034,7.2816924872863,3.20576347609776,7.78479083747996,6.65052828191008,4.91153840716089,...,5.82946940724254,4.52555330802073,4.28495973319344,3.84450065397077,4.33775413234636,5.01260428705483,5.44686361881689,8.86214646373316,GSM1233085,Response
4,3.70733339888264,9.16091157590688,6.0786756627405,5.26943202289634,5.05700806576543,6.95910517659228,4.24248677540951,7.34649764334966,6.64029157341581,4.66244351200244,...,5.98155162661694,5.89772150853027,3.78784774402191,3.81990473434895,4.16684565220916,5.06708838335397,6.52426223213204,7.18528913406378,GSM1233086,Non_response


Tras rematar el formateo y composición del dataset:
* Teníamos: 1889 muestras asociadas con 26011 genes con su expresión génica
* Acabamos con: 1889 muestras asociadas con 2396 genes con su expresión génica

Aunque la reducción de genes es muy significativa, para nuestro estudio necesitamos priorizar el número de muestras para poder entrenar a nuestro modelo, aunque ello implique tener que reducir el número de genes por no disponer de datos de expresión para todas las muestras.

In [12]:
# Guardamos el dataset ya formateado para procesar
final_expression_df.to_csv("./data/CTR_Fluorouracil_expresion_and_response_dataset.csv", index=False)